Unfortunately I couldn't use my original idea of looking at the past year of trending videos on Youtube. The Youtube data API won't let you look at historical data and will instead just grab all CURRENTLY trending videos with their view count, likes, dislikes, etc.

While searching for data sets, I was able to find the UN data website. It doesn't look like there are any visualizations or dashboards already created for their data so I thought I would take a stab at it. The first thing that jumped out to me was that the UN had data sets for A LOT of countries on births by birth month and deaths by death month. Using these data sets you can see birth and death rates by country and see which countries aren't meeting replacement rates. You can also do aggregate the geographic information up to the hemisphere, continent, and global levels.

The data files are fairly large 50k+ rows each and have some redundant data in them so I made this notebook to do some data pre-processing. This code could be added to the dashboard but with the amount of data prep I had to do, I didn't think that if I added to it in the future that keeping it within the app.R file would be scalable. If this was being prepared for a dashboard for a company, I would create an ETL process to extract the data and do all the data preprocessing for me, which would make things easier. :)

In [1]:
deaths_df <- read.csv("un_deathdata.csv")
births_df <- read.csv("un_birthdata.csv")

str(deaths_df)
str(births_df)

'data.frame':	38914 obs. of  9 variables:
 $ Country.or.Area: Factor w/ 189 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ Year           : Factor w/ 95 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ Area           : Factor w/ 2 levels "","Total": 2 2 2 2 2 2 2 2 2 2 ...
 $ Month          : Factor w/ 19 levels "","April","April - June",..: 18 7 6 12 2 13 11 9 4 17 ...
 $ Record.Type    : Factor w/ 3 levels "","Data tabulated by year of occurrence",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Reliability    : Factor w/ 4 levels "","Final figure, complete",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Source.Year    : int  2019 2019 2019 2019 2019 2019 2019 2019 2019 2019 ...
 $ Value          : int  235 17 7 24 19 12 17 29 16 23 ...
 $ Value.Footnotes: Factor w/ 62 levels "","1","1,43",..: 1 1 1 1 1 1 1 1 1 1 ...
'data.frame':	50322 obs. of  9 variables:
 $ Country.or.Area: Factor w/ 215 levels "1","10","11",..: 67 67 67 67 67 67 67 67 67 67 ...
 $ Year           : Factor w/ 119 levels

In [2]:
head(deaths_df)

Country.or.Area,Year,Area,Month,Record.Type,Reliability,Source.Year,Value,Value.Footnotes
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>
Åland Islands,2017,Total,Total,Data tabulated by year of occurrence,"Final figure, complete",2019,235,
Åland Islands,2017,Total,January,Data tabulated by year of occurrence,"Final figure, complete",2019,17,
Åland Islands,2017,Total,February,Data tabulated by year of occurrence,"Final figure, complete",2019,7,
Åland Islands,2017,Total,March,Data tabulated by year of occurrence,"Final figure, complete",2019,24,
Åland Islands,2017,Total,April,Data tabulated by year of occurrence,"Final figure, complete",2019,19,
Åland Islands,2017,Total,May,Data tabulated by year of occurrence,"Final figure, complete",2019,12,


In [3]:
tail(deaths_df)

,Country.or.Area,Year,Area,Month,Record.Type,Reliability,Source.Year,Value,Value.Footnotes
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>
38909,50,The Government of Ukraine has informed the United Nations that it is not in a position to provide statistical data concerning the Autonomous Republic of Crimea and the city of Sevastopol.,,,,,NA,NA,
38910,51,"Data includes deaths resulting from births with weight 500 g and more (if weight is unknown - with length 25 cm and more, or with gestation during 22 weeks or more).",,,,,NA,NA,
38911,52,"Excluding Channel Islands (Guernsey and Jersey) and Isle of Man, shown separately, if available.",,,,,NA,NA,
38912,53,Including unknowns.,,,,,NA,NA,
38913,54,"Data tabulated by date of occurrence for England and Wales, and by date of registration for Northern Ireland and Scotland.",,,,,NA,NA,
38914,55,Excluding Indian jungle population.,,,,,NA,NA,


Both datasets come with some footnote information as well as several columns that aren't very useful to us, at least not for the initial dashboard. We're going to first get rid of the trailing information and then the columns we don't need.

In [4]:
library(dplyr)

deaths_df <- subset(deaths_df,!is.na(Value))
deaths_df$Area <- NULL
deaths_df$Record.Type <- NULL
deaths_df$Reliability <- NULL
deaths_df$Source.Year <- NULL
deaths_df$Value.Footnotes <- NULL

births_df <- subset(births_df,!is.na(Value))
births_df$Area <- NULL
births_df$Record.Type <- NULL
births_df$Reliability <- NULL
births_df$Source.Year <- NULL
births_df$Value.Footnotes <- NULL

tail(deaths_df)
str(deaths_df)
tail(births_df)
str(births_df)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



,Country.or.Area,Year,Month,Value
,<fct>,<fct>,<fct>,<int>
38853,Wallis and Futuna Islands,2008,July,9
38854,Wallis and Futuna Islands,2008,August,4
38855,Wallis and Futuna Islands,2008,September,7
38856,Wallis and Futuna Islands,2008,October,12
38857,Wallis and Futuna Islands,2008,November,3
38858,Wallis and Futuna Islands,2008,December,5


'data.frame':	38858 obs. of  4 variables:
 $ Country.or.Area: Factor w/ 189 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ Year           : Factor w/ 95 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ Month          : Factor w/ 19 levels "","April","April - June",..: 18 7 6 12 2 13 11 9 4 17 ...
 $ Value          : int  235 17 7 24 19 12 17 29 16 23 ...


,Country.or.Area,Year,Month,Value
,<fct>,<fct>,<fct>,<dbl>
50250,Wallis and Futuna Islands,1973,July,415
50251,Wallis and Futuna Islands,1973,August,364
50252,Wallis and Futuna Islands,1973,September,300
50253,Wallis and Futuna Islands,1973,October,385
50254,Wallis and Futuna Islands,1973,November,344
50255,Wallis and Futuna Islands,1973,December,333


'data.frame':	50255 obs. of  4 variables:
 $ Country.or.Area: Factor w/ 215 levels "1","10","11",..: 67 67 67 67 67 67 67 67 67 67 ...
 $ Year           : Factor w/ 119 levels "1967","1968",..: 51 51 51 51 51 51 51 51 51 51 ...
 $ Month          : Factor w/ 19 levels "","April","April - June",..: 18 7 6 12 2 13 11 9 4 17 ...
 $ Value          : num  279 25 23 19 19 26 31 21 24 24 ...


The data also has an record for each year with the sum of the value for that year. Lets remove those rows as we can do that via dplyr.

In [5]:
deaths_df <- subset(deaths_df,Month!="Total")
births_df <- subset(births_df,Month!="Total")

str(deaths_df)
str(births_df)

'data.frame':	35907 obs. of  4 variables:
 $ Country.or.Area: Factor w/ 189 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ Year           : Factor w/ 95 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ Month          : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ Value          : int  17 7 24 19 12 17 29 16 23 26 ...
'data.frame':	46411 obs. of  4 variables:
 $ Country.or.Area: Factor w/ 215 levels "1","10","11",..: 67 67 67 67 67 67 67 67 67 67 ...
 $ Year           : Factor w/ 119 levels "1967","1968",..: 51 51 51 51 51 51 51 51 51 51 ...
 $ Month          : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ Value          : num  25 23 19 19 26 31 21 24 24 27 ...


Lets also rename our variables to maintain our preferred style guide.

In [6]:
deaths_df <- deaths_df %>%
    rename(
        country = Country.or.Area,
        year = Year,
        month = Month,
        value = Value
    )

births_df <- births_df %>%
    rename(
        country = Country.or.Area,
        year = Year,
        month = Month,
        value = Value
    )

str(deaths_df)
str(births_df)

'data.frame':	35907 obs. of  4 variables:
 $ country: Factor w/ 189 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ year   : Factor w/ 95 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ month  : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ value  : int  17 7 24 19 12 17 29 16 23 26 ...
'data.frame':	46411 obs. of  4 variables:
 $ country: Factor w/ 215 levels "1","10","11",..: 67 67 67 67 67 67 67 67 67 67 ...
 $ year   : Factor w/ 119 levels "1967","1968",..: 51 51 51 51 51 51 51 51 51 51 ...
 $ month  : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ value  : num  25 23 19 19 26 31 21 24 24 27 ...


We also want to eventually bind our data frames together to create on unified data frame so lets add a data type column so we know what the value is.

In [7]:
deaths_df$dataType <- "Deaths"
births_df$dataType <- "Births"

df <- rbind(deaths_df,births_df)

str(df)

'data.frame':	82318 obs. of  5 variables:
 $ country : Factor w/ 218 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ year    : Factor w/ 137 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ month   : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ value   : num  17 7 24 19 12 17 29 16 23 26 ...
 $ dataType: chr  "Deaths" "Deaths" "Deaths" "Deaths" ...


Perfect! Now that we've got one data frame, we can do a couple of things. First, we'll need to create a data time column in order to visualize the data as a time series. We'll also want to generate a "Continent" and "Hemisphere" column so we can do aggregations at those geographic levels. First lets look at the unique values we have for country, of which there are a lot.

In [8]:
unique(df$country)

[1] Åland Islands                                       
  [2] Albania                                             
  [3] American Samoa                                      
  [4] Andorra                                             
  [5] Anguilla                                            
  [6] Armenia                                             
  [7] Aruba                                               
  [8] Australia                                           
  [9] Austria                                             
 [10] Azerbaijan                                          
 [11] Bahamas                                             
 [12] Bahrain                                             
 [13] Barbados                                            
 [14] Belarus                                             
 [15] Belgium                                             
 [16] Bermuda                                             
 [17] Bosnia and Herzegovina                              
 [18] Brazil                                              
 [19] British Virgin Islands                              
 [20] Brunei Darussalam                                   
 [21] Bulgaria                                            
 [22] Cabo Verde                                          
 [23] Canada                                              
 [24] Cayman Islands                                      
 [25] Chile                                               
 [26] China, Hong Kong SAR                                
 [27] China, Macao SAR                                    
 [28] Cook Islands                                        
 [29] Costa Rica                                          
 [30] Croatia                                             
 [31] Cuba                                                
 [32] Curaçao                                             
 [33] Cyprus                                              
 [34] Czechia                                             
 [35] Denmark                                             
 [36] Dominica                                            
 [37] Egypt                                               
 [38] El Salvador                                         
 [39] Estonia                                             
 [40] Faeroe Islands                                      
 [41] Falkland Islands (Malvinas)                         
 [42] Fiji                                                
 [43] Finland                                             
 [44] France                                              
 [45] French Guiana                                       
 [46] French Polynesia                                    
 [47] Georgia                                             
 [48] Germany                                             
 [49] Gibraltar                                           
 [50] Greece                                              
 [51] Greenland                                           
 [52] Guadeloupe                                          
 [53] Guam                                                
 [54] Guatemala                                           
 [55] Guernsey                                            
 [56] Hungary                                             
 [57] Iceland                                             
 [58] Iran (Islamic Republic of)                          
 [59] Ireland                                             
 [60] Isle of Man                                         
 [61] Israel                                              
 [62] Italy                                               
 [63] Japan                                               
 [64] Jersey                                              
 [65] Kazakhstan                                          
 [66] Kuwait                                              
 [67] Kyrgyzstan                                          
 [68] Latvia                                     

In [10]:
#we can map each country to one contintent...
df <- df %>%
    mutate (
        continent = case_when(
            country %in% c('Åland Islands','Albania','Andorra','Austria','Belarus','Belgium','Bosnia and Herzegovina',
                          'Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Faeroe Islands','Finland','France',
                          'Germany','Gibraltar','Greece','Greenland','Guernsey','Hungary','Iceland','Ireland','Isle of Man',
                          'Italy','Jersey','Latvia','Liechtenstein','Lithuania','Luxembourg','Malta','Montenegro','Netherlands',
                          'North Macedonia','Norway','Poland','Portugal','Republic of Moldova','Romania','San Marino',
                          'Serbia','Slovakia','Slovenia','Spain','Sweden','Switzerland','Turkey','Ukraine',
                          'United Kingdom of Great Britain and Northern Ireland') ~ 'Europe',
            country %in% c('Anguilla','Antigua and Barbuda','Bahamas','Barbados','Bermuda', 'British Virgin Islands',
                          'Canada','Cayman Islands','Costa Rica','Cuba','Guadeloupe','Guatemala','Jamaica','Martinique',
                          'Mexico','Montserrat','Puerto Rico','Saint Kitts and Nevis','Saint Lucia','Saint Pierre and Miquelon',
                          'Saint Vincent and the Grenadines','Turks and Caicos Islands','United States of America',
                          'United States Virgin Islands','Dominica') ~ 'North America',
            country %in% c('Argentina','Aruba','Brazil','Chile','Curaçao','El Salvador', 'Falkland Islands (Malvinas)',
                          'French Guiana','Guyana','Panama','Peru','Suriname','Trinidad and Tobago','Uruguay','Venezuela (Bolivarian Republic of)'
                          ) ~ 'South America',
            country %in% c('Algeria','Botswana','Cabo Verde','Egypt','Ghana','Libya','Malawi','Mali','Mauritius','Reunion',
                          'Sao Tome and Principe','Seychelles','South Africa','Tunisia') ~ 'Africa',
            country %in% c('Armenia','Azerbaijan','Bahrain','Brunei Darussalam','China, Hong Kong SAR',
                          'China, Macao SAR','Georgia','Iran (Islamic Republic of)','Israel','Japan','Jordan','Kazakhstan',
                          'Kuwait','Kyrgyzstan','Lebanon','Maldives','Malaysia','Mongolia','Oman','Pakistan','Philippines','Qatar',
                          'Republic of Korea','Russian Federation','Singapore','Sri Lanka','Tajikistan','Thailand','Uzbekistan',
                          'Syrian Arab Republic') ~ 'Asia',
            country %in% c('American Samoa','Australia','Cook Islands','Fiji','French Polynesia','Guam','New Caledonia',
                          'New Zealand','Niue','Norfolk Island','Palau','Saint Helena ex. dep','Saint Helena ex. dep.','Saint Helena: Ascension',
                          'Samoa','Tonga','Wallis and Futuna Islands','Tokelau') ~ 'Oceania',
            TRUE ~ 'Antarctica'
        )
    )

str(df)

'data.frame':	82318 obs. of  6 variables:
 $ country  : Factor w/ 218 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ year     : Factor w/ 137 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ month    : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ value    : num  17 7 24 19 12 17 29 16 23 26 ...
 $ dataType : chr  "Deaths" "Deaths" "Deaths" "Deaths" ...
 $ continent: chr  "Europe" "Europe" "Europe" "Europe" ...


Lets double check that our mapping worked and see if we have any countries associated with Antartica.

In [11]:
nrow(df[df$continent=='Antarctica',])

[1] 0

Next, lets take our year and month columns and create one column out of them. First we'll take a look for any values outside of the standard months and years.

In [12]:
unique(df$month)
unique(df$year)

[1] January            February           March              April             
 [5] May                June               July               August            
 [9] September          October            November           December          
[13] Unknown            January - March    October - December April - June      
[17] July - September  
19 Levels:  April April - June August December February ... Unknown

[1] 2017 2016 2015 2014 2013 2012 2004 1991 1990 1989 1988 1987 1986 1985 1984
[16] 1983 1982 1981 2011 2010 2009 2008 2007 2006 2005 2003 2002 2001 2000 1999
[31] 1998 1997 1996 1993 1992 1994 1995 2018 1980 1979 1978 1976 1973 1971 1975
[46] 1974 1972 1977 1970 1969 1968 1967
137 Levels: 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 ... Till 2012 data refer to all live births occurred in Hungary. From 2013 data include the live births of women with Hungarian usual residence regardless of whether the live birth occurred in Hungary or in a foreign country, and do not include the live births of women with foreign country usual residence.

It looks like the years are very clean, which is good. We have some non-standard month values in there, so lets see what the overall volume is. If it is small enough we can likely get rid of it on this first pass.

In [13]:
test <- subset(df,month %in% c("Unknown","January - March","April - June","July - September","October - December"))

paste("Number of rows with non standard months:",nrow(test))
paste("Percent of records with non standard months:",round((nrow(test)/nrow(df))*100,2),"%")

[1] "Number of rows with non standard months: 1360"

[1] "Percent of records with non standard months: 1.65 %"

Given the small volume, we can just drop the records.

In [14]:
'%notin%' <- Negate('%in%')

df <- subset(df,month %notin% c("Unknown","January - March","April - June","July - September","October - December"))

unique(df$month)

[1] January   February  March     April     May       June      July     
 [8] August    September October   November  December 
19 Levels:  April April - June August December February ... Unknown

In [15]:
df$date <- as.Date(
    paste0(df$year,
         ifelse(df$month=="January","-01-",
            ifelse(df$month=="February","-02-",
            ifelse(df$month=="March","-03-",
            ifelse(df$month=="April","-04-",
            ifelse(df$month=="May","-05-",
            ifelse(df$month=="June","-06-",
            ifelse(df$month=="July","-07-",
            ifelse(df$month=="August","-08-",
            ifelse(df$month=="September","-09-",
            ifelse(df$month=="October","-10-",
            ifelse(df$month=="November","-11-",
            ifelse(df$month=="December","-12-","-00-")))))))))))),
         "01"),
    format="%Y-%m-%d"
)

str(df)

'data.frame':	80958 obs. of  7 variables:
 $ country  : Factor w/ 218 levels "1","10","11",..: 56 56 56 56 56 56 56 56 56 56 ...
 $ year     : Factor w/ 137 levels "1980","1981",..: 38 38 38 38 38 38 38 38 38 38 ...
 $ month    : Factor w/ 19 levels "","April","April - June",..: 7 6 12 2 13 11 9 4 17 15 ...
 $ value    : num  17 7 24 19 12 17 29 16 23 26 ...
 $ dataType : chr  "Deaths" "Deaths" "Deaths" "Deaths" ...
 $ continent: chr  "Europe" "Europe" "Europe" "Europe" ...
 $ date     : Date, format: "2017-01-01" "2017-02-01" ...


Now that we have our date, country, continent, and value fields all sorted out, lets save it as a CSV and work on our visualizations.

In [16]:
write.csv(df,file="un_data.csv")